## Load Train Model From Checkpoint

In [2]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [31]:
# trained model files and paths
files = {
    "pipeline_config_ssd" : os.path.join('eported_models','ssd_mobilnet_numberplate_region_detection','pipeline.config'),
    "label_map_ssd" : os.path.join('eported_models','ssd_mobilnet_numberplate_region_detection','label_map.pbtxt'),
    "pipeline_config_efficientdet" : os.path.join('eported_models','efficientdet_d0_ocr_numberplate','pipeline.config'),
    "label_map_efficientdet" : os.path.join('eported_models','efficientdet_d0_ocr_numberplate','label_map.pbtxt')
}
paths = {
    "checkpoint_path_ssd" : os.path.join('eported_models','ssd_mobilnet_numberplate_region_detection','checkpoint'),
    "checkpoint_path_efficientdet" : os.path.join('eported_models','efficientdet_d0_ocr_numberplate','checkpoint')
}

In [34]:
# Load pipeline config and build a detection model for number plate region detection
configs_ssd = config_util.get_configs_from_pipeline_file(files['pipeline_config_ssd'])
detection_model_ssd = model_builder.build(model_config=configs_ssd['model'], is_training=False)

# Restore checkpoint
ckpt_ssd = tf.compat.v2.train.Checkpoint(model=detection_model_ssd)
ckpt_ssd.restore(os.path.join(paths['checkpoint_path_ssd'], 'ckpt-0')).expect_partial()

@tf.function
def detect_fn_ssd(image):
    image, shapes = detection_model_ssd.preprocess(image)
    prediction_dict = detection_model_ssd.predict(image, shapes)
    detections = detection_model_ssd.postprocess(prediction_dict, shapes)
    return detections

In [35]:
# Load pipeline config and build a detection model for ocr
configs_efficientdet = config_util.get_configs_from_pipeline_file(files['pipeline_config_efficientdet'])
detection_model_efficientdet = model_builder.build(model_config=configs_efficientdet['model'], is_training=False)

# Restore checkpoint
ckpt_efficientdet = tf.compat.v2.train.Checkpoint(model=detection_model_efficientdet)
ckpt_efficientdet.restore(os.path.join(paths['checkpoint_path_efficientdet'], 'ckpt-0')).expect_partial()

@tf.function
def detect_fn_efficientdet(image):
    image, shapes = detection_model_efficientdet.preprocess(image)
    prediction_dict = detection_model_efficientdet.predict(image, shapes)
    detections = detection_model_efficientdet.postprocess(prediction_dict, shapes)
    return detections

In [159]:
import datetime
import pandas as pd
import cv2 
import uuid
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [200]:
# df = pd.DataFrame(columns= ['vehicle_number','day','date','time'])
# df.to_csv("vehicle_number_plate_record.csv",index=False)

In [36]:
category_index_ssd = label_map_util.create_category_index_from_labelmap(files['label_map_ssd'])
category_index_efficientdet = label_map_util.create_category_index_from_labelmap(files['label_map_efficientdet'])

In [29]:
# img_path = os.path.join('Tensorflow', 'workspace','images', 'test', 'iwt425.jpg')

In [201]:
def image_resize_with_padding(image):
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    old_size = image.shape[:2] # old_size is in (height, width) format
    
    if max(old_size) > 256:
        desired_size = 512
    else:
        desired_size = 256
        
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # resize the image
    resized = cv2.resize(image, (new_size[1],new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    # print(top,bottom,left,right)

    color = [255, 255, 255]
    new_im = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    
    return new_im
    

In [202]:
def ocr_predict(img,threshold):
    
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
    image_height = img.shape[0]
    image_width = img.shape[1]
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn_efficientdet(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    image_np_with_detections = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index_efficientdet,
                use_normalized_coordinates=True,
                max_boxes_to_draw=12,
                min_score_thresh=threshold,
                agnostic_mode=False)

    # print(detections)
    #get data(label, xmin, ymin, xmax, ymax)

    output = []
    for index, score in enumerate(detections['detection_scores']):
        if score < threshold:
            continue
        label = category_index[detections['detection_classes'][index] + label_id_offset]['name']
        # print(label)
        ymin, xmin, ymax, xmax = detections['detection_boxes'][index]
        output.append((label, int(xmin * image_width), int(ymin * image_height),
                       int(xmax * image_width), int(ymax * image_height)))

    df = pd.DataFrame(output, columns = ['label','xmin','ymin','xmax','ymax'])
    # df = df.sort_values(by = ['xmin','ymin'])
    df_up = df[df.ymin < (df.ymin.min()*1.2)].sort_values(by  = ['xmin'])
    df_down = df[df.ymin > (df.ymin.min()*1.2)].sort_values(by  = ['xmin'])
    df = pd.concat([df_up,df_down])
    vehicle_number = "".join(df["label"])
    
    vehicle_data = pd.read_csv("vehicle_number_plate_record.csv")
    
    current_date_time = datetime.datetime.now()
    day = current_date_time.strftime("%A")
    date = current_date_time.strftime("%d/%m/%Y")
    time = current_date_time.strftime("%I:%M:%S %p")
    data = [(vehicle_number,day,date,time)]
    # print(data)
    vehicle_df = pd.DataFrame(data, columns = ['vehicle_number','day','date','time'])
    vehicle_data = pd.concat([vehicle_data,vehicle_df], ignore_index=True)
    
    vehicle_data.to_csv("vehicle_number_plate_record.csv",index=False)
    
    
    print(vehicle_number)
    
    # save incident 
#     for l, x_min, y_min, x_max, y_max in output:

#         # if l == "NumberPlate": #label_to_look_for
#         array = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
#         image = Image.fromarray(array)

    # imgname = os.path.join('output_images',f'{str(uuid.uuid1())}.jpg')
    # to save full images with bounding boxes
    # image_np_with_detections.save(imgname, 'JPEG', icc_profile=image_np_with_detections.info.get('icc_profile'))
    if len(vehicle_number) == 0:
        cv2.imwrite(os.path.join('output_images',f'{str(uuid.uuid1())}.jpg'), image_np_with_detections)
    else:
        cv2.imwrite(os.path.join('output_images',f'{vehicle_number}.jpg'), image_np_with_detections)

In [203]:
def predict(folder_path,threshold):
    
    file_list = [file for file in os.listdir(folder_path) 
         if os.path.isfile(os.path.join(folder_path, file))]
    
    jpg_list = []
    substring = '.jpg'
    jpg_list = [string for string in file_list if substring in string]
    for jpg in jpg_list:
        
        img = cv2.imread(os.path.join(folder_path,jpg))
        image_height = img.shape[0]
        image_width = img.shape[1]
        image_np = np.array(img)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn_ssd(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index_ssd,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=10,
                    min_score_thresh=threshold,
                    agnostic_mode=False)

        # print(detections)
        #get data(label, xmin, ymin, xmax, ymax)
  
        output = []
        for index, score in enumerate(detections['detection_scores']):
            if score < threshold:
                continue
            label = category_index[detections['detection_classes'][index] + label_id_offset]['name']
            # print(label)
            ymin, xmin, ymax, xmax = detections['detection_boxes'][index]
            output.append((label, int(xmin * image_width), int(ymin * image_height),
                           int(xmax * image_width), int(ymax * image_height)))

        # save incident 
        for l, x_min, y_min, x_max, y_max in output:
            
            # if l == "NumberPlate": #label_to_look_for
            array = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            image = Image.fromarray(array)
            
            cropped_img = image.crop((x_min, y_min, x_max, y_max))
            # print(cropped_img.size)
            # imgname = os.path.join('Tensorflow', 'workspace','images',f'{str(uuid.uuid1())}.jpg')
            # cropped_img.save(imgname, 'JPEG', icc_profile=cropped_img.info.get('icc_profile'))
            

            input_ocr_image = image_resize_with_padding(cropped_img)
            
            ocr_predict(input_ocr_image,0.6)
            
            
            
            
            # df.loc[len(df)] = [datetime.datetime.now(), file_path]
            # df.to_csv(output_dir+'/results.csv', index=None)
        
        # # to save full images with bounding boxes
        # cv2.imwrite(os.path.join('Tensorflow', 'workspace','images','results_new_test',jpg), image_np_with_detections)
        
        ## to plot images
        # plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
        # plt.show()

In [206]:
folder_path = 'test_images'
predict(folder_path,0.6)

KLU1BM5673
KA04U5657
KA41EN1051
KL354199
GJ03LK0563
GJ01M7581
KL65L2203
6
GJ07DA9988
GJ23BM9629

MH03CE9253
N59A9Q1515
DL10SZ3519
UP54Z0690


In [193]:
df_up = df[df.ymin < (df.ymin.min()*1.2)].sort_values(by  = ['xmin'])
df_down = df[df.ymin > (df.ymin.min()*1.2)].sort_values(by  = ['xmin'])
df1 = pd.concat([df_up,df_down])
df1

,label,xmin,ymin,xmax,ymax
2,K,168,162,225,250
5,A,233,165,280,250
4,4,317,168,375,256
7,1,387,169,427,253
0,E,156,283,203,357
9,N,207,283,255,357
8,1,298,289,327,362
1,0,343,291,387,362
3,5,392,293,437,365
6,1,449,293,481,366
